<a href="https://colab.research.google.com/github/sadullahmath/Exercise/blob/master/Feature_Engineering_on_a_Financial_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Feature Engineering on a Financial Dataset

In [0]:
import pandas as pd

In [0]:
disp_url = 'https://raw.githubusercontent.com/PacktWorkshops/The-Data-Science-Workshop/master/Chapter12/Dataset/disp.csv'
trans_url = 'https://raw.githubusercontent.com/PacktWorkshops/The-Data-Science-Workshop/master/Chapter12/Dataset/trans.csv'
account_url = 'https://raw.githubusercontent.com/PacktWorkshops/The-Data-Science-Workshop/master/Chapter12/Dataset/account.csv'
client_url = 'https://raw.githubusercontent.com/PacktWorkshops/The-Data-Science-Workshop/master/Chapter12/Dataset/client.csv'

In [0]:
df_disp = pd.read_csv(disp_url, sep=';')
df_trans = pd.read_csv(trans_url, sep=';')
df_account = pd.read_csv(account_url, sep=';')
df_client = pd.read_csv(client_url, sep=';')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
df_trans.head()

,trans_id,account_id,date,type,operation,amount,balance,k_symbol,bank,account
0,695247,2378,930101,PRIJEM,VKLAD,700.0,700.0,NaN,NaN,NaN
1,171812,576,930101,PRIJEM,VKLAD,900.0,900.0,NaN,NaN,NaN
2,207264,704,930101,PRIJEM,VKLAD,1000.0,1000.0,NaN,NaN,NaN
3,1117247,3818,930101,PRIJEM,VKLAD,600.0,600.0,NaN,NaN,NaN
4,579373,1972,930102,PRIJEM,VKLAD,400.0,400.0,NaN,NaN,NaN


In [0]:
df_trans.shape

(1056320, 10)

In [0]:
df_account.head()

,account_id,district_id,frequency,date
0,576,55,POPLATEK MESICNE,930101
1,3818,74,POPLATEK MESICNE,930101
2,704,55,POPLATEK MESICNE,930101
3,2378,16,POPLATEK MESICNE,930101
4,2632,24,POPLATEK MESICNE,930102


In [0]:
df_trans_acc = pd.merge(df_trans, df_account, how='left', on='account_id')

In [0]:
df_trans_acc.shape

(1056320, 13)

In [0]:
df_disp.head()

,disp_id,client_id,account_id,type
0,1,1,1,OWNER
1,2,2,2,OWNER
2,3,3,2,DISPONENT
3,4,4,3,OWNER
4,5,5,3,DISPONENT


In [0]:
df_disp_owner = df_disp[df_disp['type'] == 'OWNER']

In [0]:
df_disp_owner.duplicated(subset='account_id').sum()

0

In [0]:
df_trans_acc_disp = pd.merge(df_trans_acc, df_disp_owner, how='left', on='account_id')
df_trans_acc_disp.shape

(1056320, 16)

In [0]:
df_client.head()

,client_id,birth_number,district_id
0,1,706213,18
1,2,450204,1
2,3,406009,1
3,4,561201,5
4,5,605703,5


In [0]:
df_merged = pd.merge(df_trans_acc_disp, df_client, how='left', on=['client_id', 'district_id'])
df_merged.shape

(1056320, 17)

In [0]:
df_merged.columns

Index(['trans_id', 'account_id', 'date_x', 'type_x', 'operation', 'amount',
       'balance', 'k_symbol', 'bank', 'account', 'district_id', 'frequency',
       'date_y', 'disp_id', 'client_id', 'type_y', 'birth_number'],
      dtype='object')

In [0]:
df_merged.rename(columns={'date_x': 'trans_date', 'type_x': 'trans_type', 'date_y':'account_creation', 'type_y':'client_type'}, inplace=True)

In [0]:
df_merged.head()

,trans_id,account_id,trans_date,trans_type,operation,amount,balance,k_symbol,bank,account,district_id,frequency,account_creation,disp_id,client_id,client_type,birth_number
0,695247,2378,930101,PRIJEM,VKLAD,700.0,700.0,NaN,NaN,NaN,16,POPLATEK MESICNE,930101,2873,2873,OWNER,755324.0
1,171812,576,930101,PRIJEM,VKLAD,900.0,900.0,NaN,NaN,NaN,55,POPLATEK MESICNE,930101,692,692,OWNER,NaN
2,207264,704,930101,PRIJEM,VKLAD,1000.0,1000.0,NaN,NaN,NaN,55,POPLATEK MESICNE,930101,844,844,OWNER,NaN
3,1117247,3818,930101,PRIJEM,VKLAD,600.0,600.0,NaN,NaN,NaN,74,POPLATEK MESICNE,930101,4601,4601,OWNER,NaN
4,579373,1972,930102,PRIJEM,VKLAD,400.0,400.0,NaN,NaN,NaN,77,POPLATEK MESICNE,930102,2397,2397,OWNER,NaN


In [0]:
df_merged.dtypes

trans_id              int64
account_id            int64
trans_date            int64
trans_type           object
operation            object
amount              float64
balance             float64
k_symbol             object
bank                 object
account             float64
district_id           int64
frequency            object
account_creation      int64
disp_id               int64
client_id             int64
client_type          object
birth_number        float64
dtype: object

In [0]:
df_merged['trans_date'] = pd.to_datetime(df_merged['trans_date'], format="%y%m%d")
df_merged['account_creation'] = pd.to_datetime(df_merged['account_creation'], format="%y%m%d")

In [0]:
df_merged.dtypes

trans_id                     int64
account_id                   int64
trans_date          datetime64[ns]
trans_type                  object
operation                   object
amount                     float64
balance                    float64
k_symbol                    object
bank                        object
account                    float64
district_id                  int64
frequency                   object
account_creation    datetime64[ns]
disp_id                      int64
client_id                    int64
client_type                 object
birth_number               float64
dtype: object

In [0]:
df_merged['is_female'] = (df_merged['birth_number'] % 10000) / 5000 > 1

In [0]:
df_merged['birth_number'].head()

0    755324.0
1         NaN
2         NaN
3         NaN
4         NaN
Name: birth_number, dtype: float64

In [0]:
df_merged.loc[df_merged['is_female'] == True, 'birth_number'] -= 5000

In [0]:
pd.to_datetime(df_merged['birth_number'], format="%y%m%d", errors='coerce')

0         1975-03-24
1                NaT
2                NaT
3                NaT
4                NaT
             ...    
1056315   2046-05-25
1056316   2066-11-01
1056317   2033-12-31
1056318   2022-07-20
1056319   2046-12-02
Name: birth_number, Length: 1056320, dtype: datetime64[ns]

In [0]:
df_merged['birth_number'] = df_merged['birth_number'].astype(str)
df_merged['birth_number'].head()

0    750324.0
1         nan
2         nan
3         nan
4         nan
Name: birth_number, dtype: object

In [0]:
import numpy as np
df_merged.loc[df_merged['birth_number'] == 'nan', 'birth_number'] = np.nan
df_merged['birth_number'].head()

0    750324.0
1         NaN
2         NaN
3         NaN
4         NaN
Name: birth_number, dtype: object

In [0]:
df_merged.loc[~df_merged['birth_number'].isna(), 'birth_number'] = '19' + df_merged.loc[~df_merged['birth_number'].isna(), 'birth_number']
df_merged['birth_number'].head()

0    19750324.0
1           NaN
2           NaN
3           NaN
4           NaN
Name: birth_number, dtype: object

In [0]:
df_merged.loc[~df_merged['birth_number'].isna(), 'birth_number'] = '19' + df_merged.loc[~df_merged['birth_number'].isna(), 'birth_number']
df_merged['birth_number'].head()

0    1919750324.0
1             NaN
2             NaN
3             NaN
4             NaN
Name: birth_number, dtype: object

In [0]:
df_merged['age_at_creation'] = df_merged['account_creation'] - df_merged['birth_number']

/usr/local/lib/python3.6/dist-packages/pandas/core/arrays/datetimelike.py:1268: PerformanceWarning: Adding/subtracting array of DateOffsets to DatetimeArray not vectorized
  PerformanceWarning,


TypeError: ignored